In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn

from pandas.io.data import DataReader
from sklearn.linear_model import LogisticRegression
from sklearn.lda import LDA
from sklearn.qda import QDA
from keras.models import Sequential
from keras.layers import LSTM, Dense
import random
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from itertools import product
from sklearn.ensemble import VotingClassifier
from sklearn import preprocessing
from sklearn import cross_validation
from scipy.stats import spearmanr 
import xgboost as xgb
from sklearn import cross_validation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import cross_val_score

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/io/data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 a

In [89]:
df = pd.read_csv('../output/dat_prediction 3.csv', index_col=0)

In [124]:
def dataselection(model, m=4):
    model = model.iloc[:,1:]
    
    X = np.array(model)[:,1:] 
    Y = np.array(model)[:,0] 
    
    n = len(model)
    
    X_test = X[:(n + 1) /m]
    Y_test = Y[:(n + 1) /m]
    X_train = X[(n + 1) /m:]
    Y_train = Y[(n + 1) /m:]
    return X_train, Y_train, X_test, Y_test, X, Y

X_train, Y_train, X_test, Y_test, X, Y = dataselection(df, m=4)

def classificationevaluation(Y_test, Y_pred):
    print '======================================'
    n = len(Y_test)
    print 'ar is: ', accuracy_score(list(Y_test), list(Y_pred))
    print 'er is: ', 1-accuracy_score(list(Y_test), list(Y_pred))
    diff = list(Y_test - Y_pred)

    print 'Type I error rate: ',  diff.count(1)/(n+0.0)
    print 'Type II error rate: ', diff.count(-1)/(n+0.0)

In [26]:
#RF
def randomforest():
    print "**************************************"
    tuned_parameters = {"max_depth": [7,9,11, None],
              "max_features": ['auto','sqrt','log2',None]}
    
    clf = RandomForestClassifier(n_estimators = 100)
    
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = RandomForestClassifier(**gs.best_params_)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'Random Forest'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [27]:
randomforest()

**************************************
Best parameters set found on development set:
{'max_features': 'log2', 'max_depth': 7}
Random Forest
ar is:  0.605633802817
er is:  0.394366197183
Type I error rate:  0.321931589537
Type II error rate:  0.0724346076459
adaboost
ar is:  0.567404426559
er is:  0.432595573441
Type I error rate:  0.319919517103
Type II error rate:  0.112676056338


In [28]:
#NB
def NB():
    print "**************************************"
    clf = GaussianNB()
    AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
                            
    clf = clf.fit(X_train, Y_train)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'NB'
    
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [29]:
NB()

**************************************
NB
ar is:  0.531187122736
er is:  0.468812877264
Type I error rate:  0.259557344064
Type II error rate:  0.209255533199
adaboost
ar is:  0.609657947686
er is:  0.390342052314
Type I error rate:  0.303822937626
Type II error rate:  0.0865191146881


In [30]:
#XGBoosting
def xgboostresult():
    print "**************************************"
    tuned_parameters = {"max_depth": [3,7,9,11],
                        'n_estimators': [100,200]}

    clf = xgb.XGBClassifier()
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = xgb.XGBClassifier(**gs.best_params_)
    clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'XGBoost'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    classificationevaluation(Y_test, clfboost.predict(X_test))

In [31]:
xgboostresult()

**************************************
Best parameters set found on development set:
{'n_estimators': 100, 'max_depth': 7}
XGBoost
ar is:  0.561368209256
er is:  0.438631790744
Type I error rate:  0.30583501006
Type II error rate:  0.132796780684
adaboost
ar is:  0.623742454728
er is:  0.376257545272
Type I error rate:  0.376257545272
Type II error rate:  0.0


In [85]:
def KNN():
    print "**************************************"
    tuned_parameters = {"n_neighbors": [5,7,9,11],
              "algorithm": ['auto','ball_tree','kd_tree']}
    
    clf = KNeighborsClassifier(n_neighbors=7)
    gs = GridSearchCV(clf, tuned_parameters, cv=5)
    gs.fit(X_train, Y_train)

    print("Best parameters set found on development set:")
    print(gs.best_params_)

    clf = KNeighborsClassifier(**gs.best_params_)
    #clfboost = AdaBoostClassifier(clf, n_estimators=50, learning_rate=1.0)
    #clfboost.fit(X_train, Y_train)
    
    clf = clf.fit(X_train, Y_train)
    y_pred = clf.predict(X_test)
    print '======================================'
    print 'KNN'
    classificationevaluation(Y_test, y_pred)
    print '======================================'
    print 'adaboost'
    #classificationevaluation(Y_test, clfboost.predict(X_test))
    return y_pred

In [33]:
KNN()

**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.583501006036
er is:  0.416498993964
Type I error rate:  0.323943661972
Type II error rate:  0.092555331992
adaboost


In [100]:
X_train, Y_train, X_test, Y_test, X, Y = dataselection(df, m=4)

In [102]:
# Recursive Feature Elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(X_train, Y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False False ..., False False False]
[568 130 199 ..., 591 687 698]


In [201]:
features = pd.DataFrame([])
features['feature'] = list(df.columns)[2:]
features['rank'] = rfe.ranking_

features[features['rank'] < 50]

,feature,rank
38,around,30
66,break,39
77,capit,45
93,cia,38
147,economi,31
150,elect,20
194,gas,46
259,jewish,21
297,march,43
301,massiv,42


In [202]:
#top feature seleciton I 
for i in range(10,65,5):
    chosed_features = list(features[features['rank']<i]['feature'])
    features_list = ['Date','Label.x']
    features_list.extend(chosed_features)
    X_train, Y_train, X_test, Y_test, X, Y = dataselection(df[features_list], m=4)
    KNN()
    print 'i is %s'%i

**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.551307847082
er is:  0.448692152918
Type I error rate:  0.297786720322
Type II error rate:  0.150905432596
adaboost
i is 10
**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.565392354125
er is:  0.434607645875
Type I error rate:  0.281690140845
Type II error rate:  0.15291750503
adaboost
i is 15
**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.581488933602
er is:  0.418511066398
Type I error rate:  0.255533199195
Type II error rate:  0.162977867203
adaboost
i is 20
**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.571428571429
er is:  0.428571428571
Type I error rate:  0.27162977867

In [204]:
#top feature seleciton II
for i in range(15,25):
    chosed_features = list(features[features['rank']<i]['feature'])
    features_list = ['Date','Label.x']
    features_list.extend(chosed_features)
    X_train, Y_train, X_test, Y_test, X, Y = dataselection(df[features_list], m=4)
    KNN()
    print 'i is %s'%i

**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.565392354125
er is:  0.434607645875
Type I error rate:  0.281690140845
Type II error rate:  0.15291750503
adaboost
i is 15
**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.565392354125
er is:  0.434607645875
Type I error rate:  0.27967806841
Type II error rate:  0.154929577465
adaboost
i is 16
**************************************
Best parameters set found on development set:
{'n_neighbors': 9, 'algorithm': 'auto'}
KNN
ar is:  0.56338028169
er is:  0.43661971831
Type I error rate:  0.267605633803
Type II error rate:  0.169014084507
adaboost
i is 17
**************************************
Best parameters set found on development set:
{'n_neighbors': 7, 'algorithm': 'auto'}
KNN
ar is:  0.543259557344
er is:  0.456740442656
Type I error rate:  0.263581488934
Typ

In [205]:
#top 22 features are selected

In [206]:
chosed_features = list(features[features['rank']<22]['feature'])
features_list = ['Date','Label.x']
features_list.extend(chosed_features)
X_train, Y_train, X_test, Y_test, X, Y = dataselection(df[features_list], m=4)
y_pred = KNN()

**************************************
Best parameters set found on development set:
{'n_neighbors': 11, 'algorithm': 'auto'}
KNN
ar is:  0.585513078471
er is:  0.414486921529
Type I error rate:  0.263581488934
Type II error rate:  0.150905432596
adaboost


In [209]:
n = len(y_pred)
prediction_result = pd.DataFrame([])
prediction_result['Date'] = df

array([ 0.,  1.,  1.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,
        1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
        1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,
        0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1